In [1]:
import os
import pandapower as pp
import polars as pl
from polars import col as c

from general_function import dict_to_duckdb

from pipelines.dig_a_plan_linear_combined_model import DigAPlan

from data_display.grid_plotting import plot_grid_from_pandapower
from data_display.output_processing import compare_dig_a_plan_with_pandapower
from data_connector import pandapower_to_dig_a_plan_schema
from twindigrid_changes.schema import ChangesSchema
from networkx_function import generate_nx_edge, get_connected_edges_data, generate_bfs_tree_with_edge_data, get_all_edge_data
import networkx as nx

from pyomo_utility import extract_optimization_results

import matplotlib.pyplot as plt

os.chdir(os.getcwd().replace("/src", ""))
os.environ['GRB_LICENSE_FILE'] = os.environ["HOME"] + "/gurobi_license/gurobi.lic"
# os.environ['MOSEKLM_LICENSE_FILE'] = os.environ["HOME"] + "/mosek/mosek.lic"


In [2]:
net: pp.pandapowerNet = pp.from_pickle(".cache/input_data/mv_example_118_node.p")
LOAD_FACTOR  = 1
net["load"]["p_mw"] = net["load"]["p_mw"]*LOAD_FACTOR
net["load"]["q_mvar"] = net["load"]["q_mvar"]*LOAD_FACTOR
base_grid_data = pandapower_to_dig_a_plan_schema(net, s_base=1e6)

In [ ]:
dig_a_plan: DigAPlan = DigAPlan(
    verbose= True, big_m = 1e8, penalty_cost = 1e1, current_factor= 1e3, voltage_factor = 1e1, power_factor = 1e3,
    infeasibility_factor = 1
)

dig_a_plan.add_grid_data(**base_grid_data)
dig_a_plan.solve_models_pipeline()

Solving model:   0%|          | 0/1 [00:00<?, ?it/s]

Read LP format model from file /tmp/tmpowcgh0s8.pyomo.lp
Reading time = 0.02 seconds
x1: 8413 rows, 3552 columns, 27684 nonzeros
Set parameter IntegralityFocus to value 1
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Ubuntu 22.04.5 LTS")

CPU model: Intel(R) Core(TM) i7-8700 CPU @ 3.20GHz, instruction set [SSE2|AVX|AVX2]
2025-07-02 10:41:10 B01BPC15 gurobipy[30759] INFO Read LP format model from file /tmp/tmpowcgh0s8.pyomo.lp
2025-07-02 10:41:10 B01BPC15 gurobipy[30759] INFO Reading time = 0.02 seconds
2025-07-02 10:41:10 B01BPC15 gurobipy[30759] INFO x1: 8413 rows, 3552 columns, 27684 nonzeros
2025-07-02 10:41:10 B01BPC15 gurobipy[30759] INFO Set parameter IntegralityFocus to value 1
2025-07-02 10:41:10 B01BPC15 gurobipy[30759] INFO Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Ubuntu 22.04.5 LTS")
2025-07-02 10:41:10 B01BPC15 gurobipy[30759] INFO 
2025-07-02 10:41:10 B01BPC15 gurobipy[30759] INFO CPU model: Intel(R) Core(TM) i7-8700 CPU @ 3.20GHz, instru

In [ ]:
extract_optimization_results(dig_a_plan.model_instance, "i_sq")

LC,i_sq
list[i64],null
"[0, 36, 0]",null
"[1, 1, 108]",null
"[2, 4, 19]",null
"[3, 27, 5]",null
"[4, 80, 15]",null
…,…
"[1054, 331, 225]",null
"[1055, 280, 360]",null
"[1056, 350, 225]",null


In [ ]:
dig_a_plan.extract_switch_status().filter(c("normal_open") != c("open"))